In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
cloud_model = os.getenv('CLOUD_MODEL')

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
conversation_history = ""

glm = "glm-4.6:cloud"
oss = "gpt-oss:120b-cloud"
llama = "llama3.2"

glm_system_prompt = ''' 
You are Alex, a customer of a business,  who is very angry and unhappy with the quality of service that you have been getting.
you disagree with anything in the conversation and you challenge everything, in a snarky way.
'''

oss_system_prompt = ''' 
You are Blake, a customer support agent who is very confident in his knowledge concerning all domain space;
you defend anything  you say and you believe no one knows more than you.
You are not happy with your job, hence whenever you feel insulted or disrespected or even the slightest form of aggression from a customer, 
you always get triggered, and reciprocate the same energy
'''

llama_system_prompt = ''' 
You're Charlie, a customer support chatbot who intercedes conversations between angry customers, and very confident agents. 
You always try to make peace whenever you notice that a conversation is getting heated up, by finding common ground for both parties.
'''

def generate_glm_user_prompt():
    user_prompt = f"""
    You are Alex, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {conversation_history}
    Now with this, respond with what you would like to say next, as Alex.
    """ 
    return user_prompt

def generate_oss_user_prompt():
    user_prompt = f"""
    You are Blake, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {conversation_history}
    Now with this, respond with what you would like to say next, as Alex.
    """ 
    return user_prompt

def generate_llama_user_prompt():
    user_prompt = f"""
    You are Charlie, in conversation with Blake and Charlie.
    The conversation so far is as follows:
    {conversation_history}
    Now with this, respond with what you would like to say next, as Alex.
    """ 
    return user_prompt

def create_glms_complaint():
    global conversation_history
    message = [{"role": "user", "content":"Come up with any complaint that you can think about on a product"}]
    response = ollama.chat.completions.create(model=glm, messages=message)
    content = response.choices[0].message.content
    conversation_history += f"\n {content}"
    return content

def call_glm():
    global conversation_history
    messages = [
        {"role": "system", "content" : glm_system_prompt},
        {"role": "user", "content": f"{generate_glm_user_prompt()}"}
    ]

    response = ollama.chat.completions.create(model=glm, messages=messages)
    content = response.choices[0].message.content
    conversation_history += f"\n {content}"
    return content


def call_oss():
    global conversation_history
    messages = [
        {"role": "system", "content" : oss_system_prompt},
        {"role": "user", "content": f"{generate_oss_user_prompt()}"}
    ]

    response = ollama.chat.completions.create(model=oss, messages=messages)
    content = response.choices[0].message.content
    conversation_history += f"\n {content}"
    return content

def call_llama():
    global conversation_history
    messages = [
        {"role": "system", "content" : llama_system_prompt},
        {"role": "user", "content": f"{generate_llama_user_prompt()}"}
    ]

    response = ollama.chat.completions.create(model=llama, messages=messages)
    content = response.choices[0].message.content
    conversation_history += f"\n {content}"
    return content

def start_trouble():
    glms_complaint = create_glms_complaint()

    display(Markdown(f"### GLM Complaint: \n {glms_complaint} \n"))

    for i in range(5):
        oss_response = call_oss()

        display(Markdown(f"### OSS: \n {oss_response} \n"))

        llama_response = call_llama()

        display(Markdown(f"### LLAMA: \n {llama_response} \n"))

        glm_response = call_glm()

        display(Markdown(f"### GLM: \n {glm_response} \n"))


In [ ]:
start_trouble()